## Generamos Embeddings con ESM2

### 1. Configuración del Tokenizador y Modelo 

In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

In [2]:
#Cargamos modelo y tokenizador
MODEL_NAME = "facebook/esm2_t33_650M_UR50D"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [4]:
model = AutoModel.from_pretrained(MODEL_NAME) #Descarga el modelo por primera vez
model.eval()

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 1280, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-32): 33 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=1280, out_features=1280, bias=True)
            (key): Linear(in_features=1280, out_features=1280, bias=True)
            (value): Linear(in_features=1280, out_features=1280, bias=True)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=1280, out_features=1280, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
          (dense): Linear(in_features=1280, out_features=5120, bias=True)
        )
        (output): EsmOut

### 2. Datos Entrada

In [5]:

from custom_dataloader import CustomProteinDataset
from torch.utils.data import DataLoader

CSV_PATH = 'protein_data.csv'
BATCH_SIZE = 5
dataset =CustomProteinDataset(CSV_PATH)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

### 3. Tokenizador

In [ ]:
next(iter(data_loader))

In [ ]:

inputs = tokenizer(
    next(iter(data_loader))[0], 
    padding=True, #Padding para el mismo tamaño
    truncation=True, #Recorta en caso de que sean muy largas
    return_tensors="pt" #Regresa tensores de pytorch
)

In [ ]:
with torch.no_grad():
    # Pasamos los tensores tokenizados al modelo
    outputs = model(**inputs)

In [ ]:
#Luego de pasar por la última capa tenemos  [num_secuencias, longitud_max_secuencia, dimension_emb]
outputs.last_hidden_state.shape 

In [ ]:
embedding_por_residuo = outputs.last_hidden_state
embedding_por_secuencia = embedding_por_residuo.mean(dim=1) #Promedio por cada secuencia

In [ ]:
embedding_por_secuencia.shape

In [ ]:
#Opcional si queremos usar numpy, sklearn
#embeddings_finales_numpy = embedding_por_secuencia.cpu().numpy()

# Predictor para pLDDT de Alphafold

In [6]:
from mlp import MLP_pLDDT
import torch.nn as nn
import torch.optim as optim

input_size = 1280
hidden_sizes = [1280,512,254]  # 3 capas ocultas
output_size = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model_mlp = MLP_pLDDT(input_size, hidden_sizes, output_size).to(device)
print(model_mlp)

criterion = nn.MSELoss()  # Mean Squared Error para regresión
optimizer = optim.Adam(model_mlp.parameters(), lr=0.001)

MLP_pLDDT(
  (network): Sequential(
    (0): Linear(in_features=1280, out_features=1280, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=1280, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=254, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=254, out_features=1, bias=True)
  )
)


In [ ]:
#MODELO DE MULTILAYER PERCEPTRON
EPOCHS = 11
for epoch in range(EPOCHS):
    total_epoch_loss = 0.0
    for i , (sequences_batch, plddts_batch) in enumerate(data_loader):
        plddts_batch = plddts_batch.to(device)
        inputs = tokenizer(sequences_batch, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        
        embeddings_batch = outputs.last_hidden_state.mean(dim=1)
        predictions = model_mlp(embeddings_batch)
        loss = criterion(predictions, plddts_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epoch_loss += loss.item()

    avg_epoch_loss = total_epoch_loss / len(data_loader)
    print(f"ÉPOCA {epoch + 1} / {EPOCHS} FINALIZADA")
    print(f"Pérdida Promedio de la Época: {avg_epoch_loss:.4f}")



In [7]:
#MODELO DE CONVOLUCIÓN 1 DIMENSIÓN
from conv1d_protein import Simple1CNN
model_cnn = Simple1CNN().to(device)
optimizer = optim.Adam(model_cnn.parameters(), lr=0.0001)
EPOCHS = 30

for epoch in range(EPOCHS):
    total_epoch_loss = 0.0
    model_cnn.train()
    for i , (sequences_batch, plddts_batch) in enumerate(data_loader):
        plddts_batch = plddts_batch.to(device)
        inputs = tokenizer(sequences_batch, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        
        embeddings_batch = outputs.last_hidden_state #Diferencia con el método anterior
        predictions = model_cnn(embeddings_batch)
        loss = criterion(predictions, plddts_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epoch_loss += loss.item()

    avg_epoch_loss = total_epoch_loss / len(data_loader)
    print(f"ÉPOCA {epoch + 1} / {EPOCHS} FINALIZADA")
    print(f"Pérdida Promedio de la Época: {avg_epoch_loss:.4f}")



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ÉPOCA 1 / 30 FINALIZADA
Pérdida Promedio de la Época: 6763.7647
ÉPOCA 2 / 30 FINALIZADA
Pérdida Promedio de la Época: 6106.0971
ÉPOCA 3 / 30 FINALIZADA
Pérdida Promedio de la Época: 5296.2391
ÉPOCA 4 / 30 FINALIZADA
Pérdida Promedio de la Época: 4394.8662
ÉPOCA 5 / 30 FINALIZADA
Pérdida Promedio de la Época: 3436.1996
ÉPOCA 6 / 30 FINALIZADA
Pérdida Promedio de la Época: 2532.4948
ÉPOCA 7 / 30 FINALIZADA
Pérdida Promedio de la Época: 1754.0055
ÉPOCA 8 / 30 FINALIZADA
Pérdida Promedio de la Época: 1141.1492
ÉPOCA 9 / 30 FINALIZADA
Pérdida Promedio de la Época: 695.5773
ÉPOCA 10 / 30 FINALIZADA
Pérdida Promedio de la Época: 438.5621
ÉPOCA 11 / 30 FINALIZADA
Pérdida Promedio de la Época: 286.6568
ÉPOCA 12 / 30 FINALIZADA
Pérdida Promedio de la Época: 204.1496
ÉPOCA 13 / 30 FINALIZADA
Pérdida Promedio de la Época: 167.7017
ÉPOCA 14 / 30 FINALIZADA
Pérdida Promedio de la Época: 156.6517
ÉPOCA 15 / 30 FINALIZADA
Pérdida Promedio de la Época: 140.8534
ÉPOCA 16 / 30 FINALIZADA
Pérdida Promedio

# Predictor para ipTM de Alphafold